# Parameter estimation with Markov chain Monte Carlo
http://bebi103.caltech.edu.s3-website-us-east-1.amazonaws.com/2015/tutorials/t4b_param_est_with_mcmc.html

In [ ]:
import collections
import itertools

# Our numerical workhorses
import numpy as np
import pandas as pd
import scipy.optimize
import scipy.stats as st

# Numerical differentiation packages
import numdifftools as ndt

# Our main MCMC package
import emcee

# Import pyplot for plotting
import matplotlib.pyplot as plt

# Seaborn, useful for graphics
import seaborn as sns

# Corner is useful for displaying MCMC results
import corner

# Magic function to make matplotlib inline; other style specs must come AFTER
%matplotlib inline

# This enables high res graphics inline (only use with static plots (non-Bokeh))
# SVG is preferred, but there is a bug in Jupyter with vertical lines
%config InlineBackend.figure_formats = {'png', 'retina'}

# JB's favorite Seaborn settings for notebooks
rc = {'lines.linewidth': 2, 
      'axes.labelsize': 18, 
      'axes.titlesize': 18, 
      'axes.facecolor': 'DFDFE5'}
sns.set_context('notebook', rc=rc)
sns.set_style('darkgrid', rc=rc)

In this tutorial, we will learn how to use Markov chain Monte Carlo to do parameter estimation. To get the basic idea behind MCMC, imagine for a moment that we can draw samples out of the posterior distribution. This means that the probability of choosing given values of a set of parameters is proportional to the posterior probability of that set of values. If we drew many many such samples, we could reconstruct the posterior from the samples, e.g., by making histograms. That's a big thing to image: that we can draw properly weighted samples. But, it turns out that we can! That is what MCMC allows us to do.

We will discuss the theory behind this seemingly miraculous capability in lecture. For today, we will just use the fact that we can do the sampling to learn about posterior distributions in the context of parameter estimation. We will use the emcee package to do our sampling by MCMC.

In [ ]:
# Load DataFrame
df = pd.read_csv('../data/singer_et_al/singer_transcript_counts.csv',
                comment='#')

In [ ]:
fig, ax = plt.subplots(2, 2)
sp_inds = list(itertools.product([0,1], [0,1]))

for i, gene in enumerate(df.columns):
    # Build ECDF
    y = np.arange(len(df[gene])) / len(df[gene])
    x = np.sort(df[gene].values)
    
    # Plot
    ax[sp_inds[i]].plot(x, y, '.')
    ax[sp_inds[i]].text(0.7, 0.25, gene, transform=ax[sp_inds[i]].transAxes,
                       fontsize=18)
    ax[sp_inds[i]].margins(0.02)
    
# Clean up
for i in [0,1]:
    ax[1,i].set_xlabel('mRNA count')
    ax[i,0].set_ylabel('ECDF')
plt.tight_layout()

The posterior
We derived last time that the posterior distributions for a single negative binomial and double-binomial model are respectively

P(r,p∣n)P(r1,r2,p1,p2,f∣n)∝∏n∈n(n+r−1)!n!(r−1)!pr(1−p)n,∝f(n+r1−1)!n!(r1−1)!pr11(1−p1)n+(1−f)(n+r2−1)!n!(r2−1)!pr22(1−p2)n.
 
As with our quest to find the MAP by optimization, we need to code up the log posterior for MCMC. Conveniently, emcee uses the same API for the posterior definition as the scipy.optimize does. We will again specify that  p1>p2 .

In [ ]:
def log_posterior(params, n):
    """
    Log posterior for MLE of Singer data.
    """
    r, p = params
    
    # Zero probability of having p < 0 or p > 1
    if p < 0 or p > 1:
        return -np.inf
    
    # Zero probability of r < 0
    if r < 0:
        return -np.inf

    return st.nbinom.logpmf(n, r, p).sum()

    
def neg_log_posterior(params, n):
    """
    Negative log posterior for MLE of Singer data.
    """
    return -log_posterior(params, n)


def log_posterior_bimodal(params, n):
    """
    Log of posterior for linear combination of neg. binomials.
    """
    r_1, r_2, p_1, p_2, f = params
    
    if (f < 0) or (f > 1):
        return -np.inf
    
    if (r_1 < 0) or (r_2 < 0) or (p_1 < p_2) or (p_2 < 0) or (p_1 > 1):
        return -np.inf
    
    return np.log(f * st.nbinom.pmf(n, r_1, p_1)
                  + (1-f) * st.nbinom.pmf(n, r_2, p_2)).sum()


def neg_log_posterior_bimodal(params, n):
    """
    Negative log posterior for linear combination of neg. binomials.
    """
    return -log_posterior_bimodal(params, n)

In [ ]:
n_dim = 2        # number of parameters in the model (r and p)
n_walkers = 50   # number of MCMC walkers
n_burn = 500     # "burn-in" period to let chains stabilize
n_steps = 5000   # number of MCMC steps to take after burn-in

In [ ]:
np.random.seed(42)


In [ ]:
# p0[i,j] is the starting point for walk i along variable j.
p0 = np.empty((n_walkers, n_dim))
p0[:,0] = np.random.exponential(0.1, n_walkers)            # r
p0[:,1] = np.random.uniform(0, 1, n_walkers)             # p

In [ ]:
sampler = emcee.EnsembleSampler(n_walkers, n_dim, log_posterior, 
                                args=(df['Prdm14'],), threads=6)

In [ ]:
# Do burn-in
pos, prob, state = sampler.run_mcmc(p0, n_burn, storechain=False)


In [ ]:
# Sample again, starting from end burn-in state
_ = sampler.run_mcmc(pos, n_steps)

In [ ]:
print(sampler.chain.shape)
print(sampler.flatchain.shape)


In [ ]:
fig, ax = plt.subplots(2, 1, sharex=True)
for i in [0, 1]:
    ax[i].plot(sampler.chain[0,:,i], 'k-', lw=0.2)
    ax[i].plot([0, n_steps-1], 
             [sampler.chain[0,:,i].mean(), sampler.chain[0,:,i].mean()], 'r-')

ax[1].set_xlabel('sample number')
ax[0].set_ylabel('r')
ax[1].set_ylabel('p')


In [ ]:
# Get the index of the most probable parameter set
max_ind = np.argmax(sampler.flatlnprobability)

# Pull out values.
r_MAP, p_MAP = sampler.flatchain[max_ind,:]

# Print the results
print("""
Most probable parameter values:
r:  {0:.3f}
p: {1:.3f}
""".format(r_MAP, p_MAP))


In [ ]:
# Compute error bars by taking standard deviation
r_err, p_err = sampler.flatchain.std(axis=0)

print('Error bars:\n', r_err, p_err)


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(9, 3))

for i in [0, 1]:
    # Plot the histogram as a step plot
    _ = ax[i].hist(sampler.flatchain[:,i], bins=100, normed=True, 
                   histtype='step', lw=2)

ax[0].set_xlabel('r')
ax[1].set_xlabel('p')
ax[0].set_ylabel('probability')


In [ ]:
fig = corner.corner(sampler.flatchain, labels=['r', 'p'], bins=100)


In [ ]:
# Parameters and how we start them
params = collections.OrderedDict(
        [('r1', (np.random.exponential, (1,))),
         ('r2', (np.random.exponential, (1,))),
         ('p1', (np.random.uniform, (0, 1))),
         ('p2', (np.random.uniform, (0, 1))),
         ('f', (np.random.uniform, (0, 1)))])

# Define walker settings
n_dim = len(params)
n_walkers = 50
n_burn = 500
n_steps = 5000

# Keep a set of param names handy
param_names = list(params.keys())

# Seed random number generator for reproducibility
np.random.seed(42)

# p0[i,j] is the starting point for walk i along variable j.
p0 = np.empty((n_walkers, n_dim))
for i, key in enumerate(params):
    p0[:,i] = params[key][0](*(params[key][1] + (n_walkers,)))

# Make sure p1 > p2
p0[:,2], p0[:,3] = np.maximum(p0[:,2], p0[:,3]), np.minimum(p0[:,2], p0[:,3])

# Set up the EnsembleSampler instance
sampler = emcee.EnsembleSampler(n_walkers, n_dim, log_posterior_bimodal, 
                                args=(df['Rex1'],), threads=6)

# Do burn-in
pos, prob, state = sampler.run_mcmc(p0, n_burn, storechain=False)

# Sample again, starting from end burn-in state
_ = sampler.run_mcmc(pos, n_steps)


In [ ]:
# Use triangle.corner to make summary plot
fig = corner.corner(sampler.flatchain, labels=param_names, bins=100)
